# Analyzing SDSS Images with Astroquery and Photutils

## Introduction

In this exercise, you will use the SDSS (Sloan Digital Sky Survey) database to retrieve images of selected galaxies and measure their angular sizes. This process involves two main Python libraries: **Astroquery** for retrieving data from the SDSS, and **Photutils** for analyzing the images and measuring the angular sizes of the galaxies.

## Retrieving SDSS Images with Astroquery

**Astroquery** is a Python library designed to simplify the process of querying astronomical databases. It allows you to retrieve data from various astronomical surveys, including SDSS.

1. **Set Up Your Environment**: Ensure you have Astroquery installed. If not, you can install it using pip:

In [1]:
!pip install astroquery photutils

2. **Query the SDSS Database**: Using Astroquery, you can search for images of the galaxies you are interested in. You will need to know the coordinates of each galaxy (right ascension and declination). Here’s how you can query the SDSS for images:

In [2]:
from astroquery.sdss import SDSS
from astropy.coordinates import SkyCoord
import astropy.units as u

# Replace these coordinates with the ones for your galaxies
coordinates = SkyCoord('0h8m05.63s +14d50m23.3s', unit=(u.hourangle, u.deg))

# Query SDSS for the image
xid = SDSS.query_region(coordinates, spectro=True, radius=0.1 * u.arcsec)

This code will retrieve data related to the galaxy's position. Adjust the parameters according to the specific galaxies you are studying.

3. **Download the Images**: Once you have the data, you can download the images. Here’s how you can do it:

In [3]:
from astropy.wcs import WCS

# Retrieve the image data from the SDSS catalog
raw_image_data = SDSS.get_images(matches=xid)

# Get the WCS information
wcs = WCS(raw_image_data[0][0].header)

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


This will provide you with the image data that you can then use for further analysis.

## Measuring Angular Size with Photutils

**Photutils** is a Python package used for detecting and measuring astronomical sources in images. It provides tools to measure the angular size of celestial objects.

1. **Load the Image**: After downloading the image with Astroquery, load it into Photutils for analysis. The image will typically be in a format that can be loaded with Astropy's fits module:

In [4]:
# Load the image. The `raw_image_data` is a list of found objects in the SDSS database. We want the first FITS file in the list, and want the first extension of that FITS file. The extension contains both header and data properties, but we just want the data. 
image_data = raw_image_data[0][0].data

2. **Detect Sources**: Use Photutils to detect the galaxy in the image. The detect_sources function can be used to identify the sources in the image.

In [94]:
from photutils.segmentation import SourceCatalog
from astropy.stats import sigma_clipped_stats
from photutils.segmentation import SourceFinder
from astropy.nddata import Cutout2D
from photutils.morphology import data_properties
from photutils.background import Background2D, MedianBackground
from astropy.convolution import convolve
from photutils.segmentation import make_2dgaussian_kernel

# The SDSS image is quite large, so we create a "cut out" of the area around the galaxy we care about.
cutout = Cutout2D(image_data, coordinates, (1 * u.arcmin, 1 * u.arcmin), wcs=wcs)
new_data = cutout.data
new_wcs = cutout.wcs

# Background estimation
bkg_estimator = MedianBackground()
bkg = Background2D(new_data, (50, 50), filter_size=(3, 3),
                   bkg_estimator=bkg_estimator)
new_data = new_data - bkg.background
threshold = 1.5 * bkg.background_rms

# Convolve the data
kernel = make_2dgaussian_kernel(3.0, size=5)
convolved_data = convolve(new_data, kernel)

# We use a built-in source finding algorithm that will identify the individual objects in the image data. We then segment the image based on these objects.
finder = SourceFinder(npixels=10, progress_bar=False)
segment_map = finder(convolved_data, threshold)

# From the segmentation map, we can create a source catalog that contains the properties of the detected sources.
cat = SourceCatalog(new_data, segment_map, wcs=new_wcs, convolved_data=convolved_data)[0]

# Turn the catalog into a table for easier viewing.
source_table = cat.to_table()
source_table.to_pandas()

,label,xcentroid,ycentroid,sky_centroid.ra,sky_centroid.dec,bbox_xmin,bbox_xmax,bbox_ymin,bbox_ymax,area,...,semiminor_sigma,orientation,eccentricity,min_value,max_value,local_background,segment_flux,segment_fluxerr,kron_flux,kron_fluxerr
0,1,75.771484,75.802379,2.023436,14.839822,44,108,51,101,1980.0,...,5.944968,36.326232,0.875251,-0.02594,5.513583,0.0,334.383573,NaN,345.921199,NaN


4. **Measure Angular Size**: Measure the angular size of the detected galaxy. Photutils provides several tools to compute properties like the semi-major and semi-minor axes of the galaxy, which are related to its angular size.

In [ ]:
import matplotlib.pyplot as plt
from photutils.aperture import EllipticalAperture
from astropy.visualization import simple_norm

norm = simple_norm(new_data, 'sqrt')

f, ax = plt.subplots(subplot_kw={'projection': wcs})

r = 3.0  # approximate isophotal extent
a = cat.semimajor_sigma.value * r
b = cat.semiminor_sigma.value * r
theta = cat.orientation.to(u.rad).value

aperture = EllipticalAperture((cat.xcentroid, cat.ycentroid), a, b, theta=theta)
aperture.plot(color='r')

ax.imshow(new_data, origin='lower', norm=norm)

print(f"The diameter of the galaxy is approximately {2 * a:.2f} arcseconds.")